# Perform OLAP Operations [Drill down, Roll up, Dice, Slice, Pivot]

In [ ]:
import sqlite3 

conn = sqlite3.connect("OLAP.db") 
cursor = conn.cursor() 

# Create table with 'quantity' column
cursor.execute(""" 
CREATE TABLE IF NOT EXISTS sale ( 
    sale_id INTEGER PRIMARY KEY, 
    product_name TEXT, 
    product_price REAL, 
    quantity INTEGER, 
    year INTEGER
); 
""")

def insert_user(): 
    try: 
        sale_id = int(input("Enter sales id: "))  
        product_name = input("Enter product name: ").strip()  
        product_price = float(input("Enter Product Price: "))  
        quantity = int(input("Enter quantity sold: "))  
        year = int(input("Enter year: "))  

        cursor.execute("INSERT INTO sale (sale_id, product_name, product_price, quantity, year) VALUES (?, ?, ?, ?, ?)", 
                       (sale_id, product_name, product_price, quantity, year)) 
        conn.commit() 
        print("Record Inserted successfully!") 

    except ValueError as e: 
        print("Invalid input:", e) 
    except sqlite3.IntegrityError:
        print("Error: Sale ID already exists!") 

def drill_down():
    try:
        year = int(input("Enter the year to drill down into sales data: "))
        cursor.execute("SELECT product_name, SUM(product_price * quantity) FROM sale WHERE year = ? GROUP BY product_name", (year,))
        results = cursor.fetchall()

        if results:
            print(f"\nDrill Down - Sales in Year {year}:")
            for row in results:
                print(row)
        else:
            print("No data found for the given year.")
    except ValueError:
        print("Invalid year input.")

def roll_up():
    cursor.execute("SELECT year, SUM(product_price * quantity) FROM sale GROUP BY year")
    results = cursor.fetchall()

    print("\nRoll Up - Total Sales by Year:")
    for row in results:
        print(row)

def dice():
    try:
        year = int(input("Enter the year: "))
        product_name = input("Enter product name: ").strip()
        cursor.execute("SELECT * FROM sale WHERE year = ? AND product_name = ?", (year, product_name))
        results = cursor.fetchall()

        if results:
            print(f"\nDice - Filtering data for {product_name} in {year}:")
            for row in results:
                print(row)
        else:
            print("No matching records found.")
    except ValueError:
        print("Invalid input.")

def slice():
    try:
        year = int(input("Enter the year to slice: "))
        cursor.execute("SELECT * FROM sale WHERE year = ?", (year,))
        results = cursor.fetchall()

        if results:
            print(f"\nSlice - Sales Data for Year {year}:")
            for row in results:
                print(row)
        else:
            print("No records found for the given year.")
    except ValueError:
        print("Invalid input.")

def pivot():
    cursor.execute("SELECT product_name, year, SUM(product_price * quantity) FROM sale GROUP BY product_name, year")
    results = cursor.fetchall()

    if results:
        print("\nPivot - Sales Pivot Table:")
        for row in results:
            print(row)
    else:
        print("No sales data available.")

def main():
    while True:
        print("\nChoose an OLAP operation:")
        print("1. Drill Down")
        print("2. Roll Up")
        print("3. Dice")
        print("4. Slice")
        print("5. Pivot")
        print("6. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            drill_down()
        elif choice == '2':
            roll_up()
        elif choice == '3':
            dice()
        elif choice == '4':
            slice()
        elif choice == '5':
            pivot()
        elif choice == '6':
            break
        else:
            print("Invalid choice, please try again.")

main()

conn.close()
